In [1]:
%matplotlib inline
import sys, os, time

import numpy as np
import pandas as pd

import fiona
import rasterio
import rasterio.mask
import shapely
import shapely.geometry

import matplotlib.pyplot as plt

from collections import Counter, defaultdict

In [2]:
def humansize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [3]:
def get_random_string(n):
    alphabet = list("abcdefghijklmnopqrstuvwxyz".upper())
    return ''.join(np.random.choice(alphabet, n, replace=True))

In [4]:
def get_lc_stats(data):
    vals = [1, 2, 3, 4, 5, 6, 15]
    counts = []
    for val in vals:
        counts.append((data==val).sum())
    return np.array(counts)

In [5]:
def get_nlcd_stats(data):
    counts = []
    for val in NLCD_CLASS_IDX:
        counts.append((data==val).sum())
    return np.array(counts)

In [6]:
NLCD_CLASSES = [
    0, 11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95, 255
]
NLCD_CLASSES_TO_IDX = defaultdict(lambda: 0, {cl:i for i,cl in enumerate(NLCD_CLASSES)})
NLCD_CLASS_IDX = range(len(NLCD_CLASSES))

def bounds_intersection(bound0, bound1):
    left0, bottom0, right0, top0 = bound0
    left1, bottom1, right1, top1 = bound1
    left, bottom, right, top = \
            max([left0, left1]), max([bottom0, bottom1]), \
            min([right0, right1]), min([top0, top1])
    return (left, bottom, right, top)

In [7]:
f = fiona.open("../data/good_blobs_epsg4326.shp","r")
fns = []
for line in f:
    fns.append(line["properties"]["location"])
f.close()

In [8]:
state_year_fns = defaultdict(list)
for fn in fns:
    parts = fn.split("/")
    state_year_fns[parts[9]].append(fn)

In [9]:
for k, vs in state_year_fns.items():
    print(k, len(vs))

de_1m_2013 144
ny_1m_2013 1308
nj_1m_2013 27
va_1m_2014 2056
md_1m_2013 823
wv_1m_2014 594
pa_1m_2013 2430
md_1m_2015 9
nc_1m_2014 9


In [10]:
good_blobs_to_best_tiles_map = {}
f = open("../data/good_blobs_to_best_tiles_map.csv", "r")
f.readline()
lines = f.read().strip().split("\n")
for line in lines:
    parts = line.split(",")
    good_blobs_to_best_tiles_map[parts[0]] = parts[1]
f.close()

In [11]:
# These naip files have blacked out areas 
bad_fns = pd.read_csv("../data/naip_num_zeros.csv")
bad_fns = set(bad_fns[bad_fns.num_zeros>0].naip_fn.tolist())

In [22]:
train_state = "ny_1m_2013"
output_dir = "/mnt/blobfuse/cnn-minibatches/summer_2019/%s_train/" % (train_state)
num_tiles = 500
samples_per_tile = 200
sample_size = 240
num_channels = 6
num_bytes_per_channel = 4
average_tile_size = 7000


df = pd.read_csv("splits/%s_test_split.csv" % (train_state))
test_fns =  set(df["naip_path"].values)
all_fns = set(state_year_fns[train_state])
remaining_fns = list(all_fns - test_fns - bad_fns)
train_fns = np.random.choice(remaining_fns, min(len(remaining_fns), num_tiles), replace=False)


print("Size of all_fns", len(all_fns))
print("Size of test_fns", len(test_fns))
print("Size of remaining_fns", len(remaining_fns))
print("Size of bad_fns", len(bad_fns))
print("Size of train_fns", len(train_fns))
print("Removed %d bad_fns" % (len(all_fns) - len(remaining_fns) - len(test_fns)))
print("")
print("Fraction of available training tiles samples", num_tiles/len(remaining_fns))
print("Number of samples", num_tiles * samples_per_tile)
print("Number of samples_per_tile that will give complete coverage", (average_tile_size/sample_size) * (average_tile_size/sample_size))
print("Expected fraction of each tile that will be sampled", (samples_per_tile * (sample_size*sample_size)) / (average_tile_size*average_tile_size))
print("Size of sampled data", humansize(num_tiles * samples_per_tile * (sample_size*sample_size) * num_channels * num_bytes_per_channel))


Size of all_fns 1308
Size of test_fns 50
Size of remaining_fns 1258
Size of bad_fns 40
Size of train_fns 500
Removed 0 bad_fns

Fraction of available training tiles samples 0.397456279809221
Number of samples 100000
Number of samples_per_tile that will give complete coverage 850.6944444444445
Expected fraction of each tile that will be sampled 0.23510204081632652
Size of sampled data 128.75 GB


In [23]:
f = open("splits/%s_train_split.csv" % (train_state),"w")
f.write("naip_path,lc_path,nlcd_path\n")

for naip_fn in train_fns:
    
    lc_fn = naip_fn.replace("esri-naip", "resampled-lc")[:-4] + "_lc.tif"
    naip_fn = good_blobs_to_best_tiles_map[naip_fn]
    nlcd_fn = naip_fn.replace("esri-naip", "resampled-nlcd")[:-4] + "_nlcd.tif"
    
    f.write("%s,%s,%s\n" % (naip_fn, lc_fn, nlcd_fn))
f.close()

In [24]:
patch_fns = []

for count, naip_fn in enumerate(train_fns):
    print(count, len(train_fns))
    
    lc_fn = naip_fn.replace("esri-naip", "resampled-lc")[:-4] + "_lc.tif"
    naip_fn = good_blobs_to_best_tiles_map[naip_fn]
    nlcd_fn = naip_fn.replace("esri-naip", "resampled-nlcd")[:-4] + "_nlcd.tif"
    
    naip_f = rasterio.open(naip_fn, "r")
    naip_bounds = naip_f.bounds
    
    lc_f = rasterio.open(lc_fn, "r")
    lc_bounds = lc_f.bounds

    nlcd_f = rasterio.open(nlcd_fn, "r")
    nlcd_bounds = nlcd_f.bounds

    bounds = bounds_intersection(bounds_intersection(naip_bounds, lc_bounds), nlcd_bounds)
    left, bottom, right, top = bounds
    geom = shapely.geometry.mapping(shapely.geometry.box(left, bottom, right, top, ccw=True))

    naip_data, _ = rasterio.mask.mask(naip_f, [geom], crop=True)
    #naip_data = np.rollaxis(naip_data, 0, 3)
    naip_f.close()
    lc_data, _ = rasterio.mask.mask(lc_f, [geom], crop=True)
    #lc_data = np.squeeze(lc_data)
    lc_f.close()
    nlcd_data, _ = rasterio.mask.mask(nlcd_f, [geom], crop=True)
    nlcd_f.close()
    nlcd_data = np.vectorize(NLCD_CLASSES_TO_IDX.__getitem__)(nlcd_data).astype(np.uint8)
    
    #print(naip_fn, naip_data.shape, naip_data.dtype)
    #print(nlcd_fn, nlcd_data.shape, nlcd_data.dtype)
    #print(lc_fn, lc_data.shape, lc_data.dtype)
    
    _, height, width = naip_data.shape
    
    for i in range(samples_per_tile):
        
        y = np.random.randint(0, height-sample_size)
        x = np.random.randint(0, width-sample_size)

        merged = np.concatenate([
            naip_data[:, y:y+sample_size, x:x+sample_size].astype(np.float32) / 255.0,
            lc_data[:, y:y+sample_size, x:x+sample_size],
            nlcd_data[:, y:y+sample_size, x:x+sample_size],
        ])
        
        lc_string = '_'.join(map(str,get_lc_stats(merged[4,:,:])))
        nlcd_string = '_'.join(map(str,get_nlcd_stats(merged[5:,:])))
        
        output_fn = "%s_%s_%s-%s-%s.npy" % (
            train_state,
            os.path.basename(naip_fn)[:-4],
            get_random_string(4),
            lc_string,
            nlcd_string
        )
        
        np.save(os.path.join(output_dir, output_fn), merged[np.newaxis].data)
        patch_fns.append(os.path.join(output_dir, output_fn))

0 500
1 500
2 500
3 500
4 500
5 500
6 500
7 500
8 500
9 500
10 500
11 500
12 500
13 500
14 500
15 500
16 500
17 500
18 500
19 500
20 500
21 500
22 500
23 500
24 500
25 500
26 500
27 500
28 500
29 500
30 500
31 500
32 500
33 500
34 500
35 500
36 500
37 500
38 500
39 500
40 500
41 500
42 500
43 500
44 500
45 500
46 500
47 500
48 500
49 500
50 500
51 500
52 500
53 500
54 500
55 500
56 500
57 500
58 500
59 500
60 500
61 500
62 500
63 500
64 500
65 500
66 500
67 500
68 500
69 500
70 500
71 500
72 500
73 500
74 500
75 500
76 500
77 500
78 500
79 500
80 500
81 500
82 500
83 500
84 500
85 500
86 500
87 500
88 500
89 500
90 500
91 500
92 500
93 500
94 500
95 500
96 500
97 500
98 500
99 500
100 500
101 500
102 500
103 500
104 500
105 500
106 500
107 500
108 500
109 500
110 500
111 500
112 500
113 500
114 500
115 500
116 500
117 500
118 500
119 500
120 500
121 500
122 500
123 500
124 500
125 500
126 500
127 500
128 500
129 500
130 500
131 500
132 500
133 500
134 500
135 500
136 500
137 500
138 50

In [26]:
f = open("data/%s_train_patches.txt" % (train_state),"w")
f.write("\n".join(patch_fns))
f.close()

In [28]:
len(patch_fns)

100000

## Assert that each file exists

In [9]:
f = open("data/ny_1m_2013_all_patches.txt", "r")
patch_fns = f.read().strip().split("\n")
f.close()

In [10]:
for fn in patch_fns:
    assert os.path.exists(fn), fn